In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import time

In [ ]:
%cd /content/drive/MyDrive/Data/Muzzle_Images/Clean_2000_to_3100

In [ ]:
import os
import pandas as pd
import cv2
from scipy import spatial
from torch.utils.data import Dataset
from tqdm import tqdm
import PIL.Image
from PIL import Image
from sklearn.neighbors.unsupervised import NearestNeighbors
from sklearn.metrics import accuracy_score, confusion_matrix
from torchvision.transforms.functional import to_tensor,to_pil_image
import torch
import numpy as np
from torchvision import utils
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import torchvision.transforms.functional as TF
import random
import time
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from torchvision import models
from torch import nn
import copy
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision.utils import save_image

In [ ]:
import sys
sys.path.append("/content/drive/My Drive/Rakesh/Muzzle_Similarity/Muzzle_Detection")

from numpy import random
from models.experimental import attempt_load
from utils.datasets import  LoadImages
from utils.general import (check_img_size, non_max_suppression,  plot_one_box, scale_coords)

In [ ]:
def detect_muzzle(source="/content/drive/MyDrive/Data/Muzzle_Images/Clean_2000_to_3100/raw_images/test", weights = "weights/last.pt", conf_thres=0.4, iou_thres=0.5, view_img=0, imgsz=416, device_name="cpu"):

    # Initialize
    device = torch.device(device_name)

    half = device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    imgsz = check_img_size(imgsz, s=model.stride.max())  # check img_size
    if half:
        model.half()  # to FP16


    # Set Dataloader
    dataset = LoadImages(source, img_size=imgsz)

    # Get names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

    # Run inference
    actual_cord = []
    found = False
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
    _ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        pred = model(img, augment=False)[0]

        # Apply NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes=None, agnostic=False)

        # Process detections
        
        for i, det in enumerate(pred):  # detections per image
            p, s, im0 = path, '', im0s
            if det is not None and len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()
                tlbr = det[:, :4].cpu().numpy().astype(int)[0] # top-left, bottom-right

                actual_cord.append(tlbr)
                found =  True
                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if view_img:  # Add bbox to image
                        label = '%s %.2f' % (names[int(cls)], conf)
                        plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=3)


            # Stream results
            if view_img:
                vis_image = cv2.resize(im0, (imgsz, imgsz))
                if tlbr[0] != None:
                    muzzle_image = cv2.resize(im0[tlbr[1]:tlbr[3], tlbr[0]:tlbr[2]], (imgsz, imgsz))
                    vis_image = np.concatenate((vis_image, muzzle_image), axis=1)
                
                plt.imshow(vis_image)
                plt.title('Semi-Crop                Fully Cropped')

                tlbr = [None]
                if cv2.waitKey(1) == ord('q'):  # q to quit
                    raise StopIteration

    return found, actual_cord


In [ ]:
class ClaheTransform(object):

    def __call__(self, img):
        """
        :param img: PIL Raw Image 

        :return: CLAHE image
        """
        img = np.asarray(img)
        img_dtype = img.dtype
        img = img.astype(np.uint8)
        #-----Converting image to LAB Color model----------------------------------- 
        lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

        #-----Splitting the LAB image to different channels-------------------------
        l, a, b = cv2.split(lab)

        #-----Applying CLAHE to L-channel-------------------------------------------
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)

        #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
        limg = cv2.merge((cl,a,b))

        #-----Converting image from LAB Color model to RGB model--------------------
        final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        clahe_img = final.astype(img_dtype)

        return Image.fromarray(clahe_img)

In [ ]:
def clahe_crop(imagefile):
    
    
    img=Image.open(imagefile)
    
    # Apply CLAHE Transform
    clh_transforms = transforms.Compose([ClaheTransform()])
    img = clh_transforms(img)
    
    #Detect Muzzle
    found, coord = detect_muzzle(source=imagefile, weights="/content/drive/My Drive/Rakesh/Muzzle_Similarity/Muzzle_Detection/weights/last.pt")
    if found:
        coord  = coord[0]
        top    = coord[1]
        left   = coord[0]
        height = coord[3] - coord[1]
        width  = coord[2] - coord[0]
        img = transforms.functional.crop(img, top, left, height, width)

#     # Apply Default Transform
    def_transforms = transforms.Compose([
                                    transforms.Resize((224,224)),
#                                     transforms.ToTensor(),
#                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),# Image Net
#                                     transforms.ToPILImage()
                                    ])
    img = def_transforms(img)
    return img

In [ ]:
path2raw = "/content/drive/MyDrive/Data/Muzzle_Images/yolo_field_laptop/multiple_mobile_testing/laptop/images_test"
filenames = []
for (dirpath, dirnames, files) in os.walk(path2raw):      
    filenames += [os.path.join(dirpath, file) for file in files]


for file in filenames:
    full_crop_img = full_crop(file)
    save_path = file.replace("images_test", "images_test_full_crop")
    full_crop_img.save(save_path)